In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt

import eq

In [2]:

catalog = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=3,
    t_end_days=365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    radius_kilometers = 1000,
)

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


In [5]:
catalog.train.sequences

[Sequence(
   inter_times: [492],
   arrival_times: [491],
   t_start: 0.0,
   t_end: 365.0,
   t_nll_start: 0.0,
   mag: [491],
   mag_bounds: [2],
   mag_nll_bounds: [2],
   extra_feat: [491]
 )]

In [13]:
catalog.train

InMemoryDataset(1)

In [ ]:
dl_val = catalog.val.get_dataloader(batch_size=min(500,len(catalog.val)))
dl_train = catalog.train.get_dataloader(batch_size=min(500,len(catalog.train)))

In [ ]:
catalog_exp = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=18000,
    t_end_days=365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    radius_kilometers = 1000
)

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


In [5]:
catalog_exp.train[0]

Sequence(
  inter_times: [112],
  arrival_times: [111],
  t_start: 0.0,
  t_end: 365.0,
  t_nll_start: 0.0,
  mag: [111]
)

In [10]:


dl_train = catalog.train.get_dataloader(batch_size=1)
dl_val = catalog.val.get_dataloader(batch_size=1)
dl_test = catalog.test.get_dataloader(batch_size=1)

T = catalog.train[0].t_end
N = np.mean([len(seq) for seq in catalog.train])
mag_mean = np.mean([catalog.train[0].mag.mean().item() for seq in catalog.train])
tau_mean = T/N

model = eq.models.RecurrentTPP(
    tau_mean = tau_mean,
    learning_rate=1e-3,
)

    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.5 K 
3 | dropout  

Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:103: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  rank_zero_warn(
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the nu

Training: 0it [00:00, ?it/s]

RuntimeError: Early stopping conditioned on metric `total_val_loss` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train_loss`, `total_train_loss`

In [29]:
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)

RecurrentTPP(
  (hypernet_time): Linear(in_features=32, out_features=96, bias=True)
  (hypernet_mag): Linear(in_features=32, out_features=1, bias=True)
  (rnn): GRU(2, 32, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [30]:
trainer.test(model, dl_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -2.942216157913208     │
│      total_test_loss      │    -2.942216157913208     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -2.942216157913208, 'total_test_loss': -2.942216157913208}]